**Kafka:** büyük verileri anlık olarak depolamak ve analiz etmek için kullanılan açık kaynak bir framework’dür. Büyük verileri hızlı bir şekilde depolayıp analiz etmek için ise mesajlaşma sistemini (queue) kullanmaktadır. 

    Kafka Topic,Producer ve Consumer kavramlarına sahiptir.
    
    Topicler kullanıcı tarafından isimlendirilebilen, verilerin (mesajların) depolandığı alandır.
    
    Topicler Partitionlara ayrılmıştır ve depolandığı Partition sayısı kullanıcı tarafından belirlenebilir. Producerler ise bu Topiclere mesaj gönderebilen Apache Kafka’nın Publish-Subscribe yapısına baktığımızda Publisher konumundadır. Topicler içerisine veri yani mesaj gönderebilirler ve aynı anda birden fazla konuya bağlı olabilirler. Consumer ise adından anlayabileceğimiz gibi Producerlar tarafından Topiclere gönderilen mesajları tüketen Subscriberlardır. Bir konuya (Topic) birden fazla Producer mesaj gönderebilir ve yine bir konuya birden fazla Consumer dahil olup Topic içerisine gönderilen verileri okuyabilir, producerların gönderdiği bu mesajları bir consumer okuduktan sonra bu veriler Topic içerisinden silinmez . Konu başında verdiğimiz örnek ile daha iyi anlayabilmemiz için bu yapıya göre Consumer kullanıcı loglarını toplayan şirket, Producer ise şirkete veri gönderen kullanıcılar konumundadır.

![title](img/kafka.png)


**Partition**
Topicler görselde görüldüğü gibi Partitionlardan oluşur ve Topic içerisindeki veriler bu Partitionlarda sıralı ve değiştirilemez olarak eklenirler ve artan şekilde bir offset değeri alırlar. Topic’in depolanacağı bu Partition sayısı kullanıcı tarafından belirlenebilir.

**Brokers**
Brokerlar Topic ve Partitionları tutan sunuculardır. Brokerlar Topiclerin depolandığı Partitionları içerir ve eğer aynı sayıda Broker ve Partition varsa her Broker bünyesinde bir Partition bulundurur. Partition sayısı Broker sayısını geçtiğinde ise bu sayı eşit olarak Brokerlar arasında dağıtılır. Bir kafka Broker grubuna ise Kafka Cluster denir. Her Broker içerisinde, bir server Lider seçilir, diğer serverlar follower olarak adlandırılır ve içerisinde verilerin kopyasını tutarlar. Herhangi bir sorun durumunda bu follewerların kendisini Lider olarak atama yeteneği vardır ve bu sayede veri akışı kesintisiz olarak devam eder. Kopya alınacak server sayısını ise kullanıcı belirleyebiliyor ve elbette ki kopya sayısı performansa olumlu-olumsuz etki edebiliyor.


**Ønemli kisa notlar**
    
   **Distributed streaming platform**
    
    1-Kurum/sirket mesaj kuyrugu(message queue) veya mesaj sistemine benzer 
    2-yayinla(publish)-abone ol (subscribe) matigiyla calisir.
    3-Mesajlari hataya dayanili bir sekilde bellir bir sure saklar ama kesinlikle bir veri tabani degildir.
    4-Kendisine birakilan mesaji oldugu gibi iletir,degistirmez
    5-cluster halinde calisir yani yatay olceklendirilebilir.
    
**Temel Kavramlari**
     
    *Producer: Kafkaya mesaj gonderen uygulamaya denir.(meaaj kafkanin uyguladigi en kucuk parca)
    *Consumer:KAfka ya gelip mesaj okuyan uygulama
    *Broker: Cluster icinde kafkaya verilen addir.(Clusterin her bir sunucusuna denir)
    *Cluster:
    *Topic: Topic icin tablo ve mesaj icin satir benzetmesi yapabiliriz
    *Partitions:Topicleri partitions lara ayirir(hataya karsi dayanikli olabilmesi icin)Performans,disk buyuklugu
    *Offset:onceden okudugu yere kadarki poziyonu gormek icin okudgu yerlerin basina 0 yaziyor
    *Consumer groups:
    
 *Bir mesaja erismek icin **Topic numarasi,Partition,offset** bilmek gerekiyor*
 
 *Hataya karsi Dayaniklilik(Fault Tolerance) ,**replikasyon**(topic seviyesinde belirlenir) numarasi eklenip,**partitions**'a uygulanir* Ayni replikaya sahip sunucular veriyi kaybetmeden hizmeti devam ettiriyor
 
**Leader Replica** 

# READ FROM KAFKA

In [ ]:
#installation guide
#-https://www.apache.org/dyn/closer.cgi?path=/kafka/2.5.0/kafka_2.12-2.5.0.tgz(1)
#-https://kafka.apache.org/quickstart

In [ ]:
import findspark
findspark.init("/Users/resitkadir/spark/spark-3.0.0")
from warnings import filterwarnings
filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'


spark = SparkSession.builder \
        .master("local[4]") \
        .appName("ReadFROMKafka") \
        .getOrCreate()

df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers","localhost:9092") \
        .option("subscribe","test1") \
        .load()
#key value seklinde calisir
df2 = df.selectExpr(" CAST(key AS STRING)", "CAST(value AS STRING)")

#words leri alalim
#bosluklardan ayirim yeni satira value ismini vereleim
#flatmap= explode ile ayni isi yapar split ile

words = df2.select(explode(split(col("value"), " ")).alias("value"))

#simdi sayalim,grouoby ile
word_conuts = words.groupBy("value").count().sort(desc("count"))


#aggregate edilmis data datamizi alalim
#jupyter consolena yaz
#Complete cunku aggregate 
query = word_conuts.writeStream \
        .format("console") \
        .outputMode("complete") \
        .start()
        #.trigger(processingTime='1 seconds') \
#query = word_conuts.writeStream 

#sumdi jupyter consolde kontrol edebiliriz



#mesaji gondermek icin kafka da terminale yazdir
#producer dan yolluyoruz mesajlari
#->bin/kafka-console-producer.sh --bootstrap-server localhost:9092 --topic test
query.awaitTermination()

In [1]:
import findspark
findspark.init("/Users/resitkadir/spark/spark-3.0.0")
from warnings import filterwarnings
filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'


spark = SparkSession.builder \
        .master("local[4]") \
        .appName("WriteToKafka") \
        .getOrCreate()

df = spark.read.format("csv") \
    .option("header",True) \
    .load("datasets/Advertising.csv")

df.show(3)

+---+-----+-----+---------+-----+
|_c0|   TV|Radio|Newspaper|Sales|
+---+-----+-----+---------+-----+
|  1|230.1| 37.8|     69.2| 22.1|
|  2| 44.5| 39.3|     45.1| 10.4|
|  3| 17.2| 45.9|     69.3|  9.3|
+---+-----+-----+---------+-----+
only showing top 3 rows



In [2]:
#simdi kafka ya yazalim
#kafkanin calisma mantigi key value seklindedir (datayi key value cevirmeliyiz)
#sutun birlestirme lit dir
df2 = df.withColumn("key",col("_c0")).drop("_c0")
df3 = df2.select("key", concat(
                        col("TV"),lit(","),
                        col("Radio"),lit(","),
                        col("Newspaper"),lit(","),
                        col("Sales")).alias("value")
                             )
df3.show(2)

+---+--------------------+
|key|               value|
+---+--------------------+
|  1|230.1,37.8,69.2,22.1|
|  2| 44.5,39.3,45.1,10.4|
+---+--------------------+
only showing top 2 rows



In [3]:
df3 \
.write \
.format("kafka") \
.option("kafka.bootstrap.servers","localhost:9092") \
.option("topic","test") \
.save()

Py4JJavaError: An error occurred while calling o57.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3, resits-mbp, executor driver): org.apache.kafka.common.errors.TimeoutException: Topic test not present in metadata after 60000 ms.

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2023)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:1972)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:1971)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1971)
	at org.apache.spark.scheduler.DAGSched

In [2]:
#simdi kafka ya yazalim
#kafkanin calisma mantigi key value seklindedir (datayi key value cevirmeliyiz)
#sutun birlestirme lit dir
df2 = df.withColumn("key",col("_c0")).drop("_c0")
df3 = df2.select("key", concat(
                        col("TV"),lit(","),
                        col("Radio"),lit(","),
                        col("Newspaper"),lit(","),
                        col("Sales")).alias("value")
                             )
df3.show(2)

+---+--------------------+
|key|               value|
+---+--------------------+
|  1|230.1,37.8,69.2,22.1|
|  2| 44.5,39.3,45.1,10.4|
+---+--------------------+
only showing top 2 rows



In [ ]:
df3 \
.write \
.format("kafka") \
.option("kafka.bootstrap.servers","localhost:9092") \
.option("topic","test") \
.save()